# Multiple Linear Regression — Forecasting Survival_Months



**Course:** CDS 2413 — Programming for Data Science  
**CRN:** 14350  
**Project:** Cancer Outcome Classification (UAE Synthetic Records)  
**Group:** 3  
**Members:** Aisha Alshamsi (H00535685), Sara Almulla (H00532724)  
**Dataset Size:** 10,000 records

In [2]:

import pandas as pd, numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

df = pd.read_csv("cancer_dataset_uae.csv")
for c in ["Diagnosis_Date","Death_Date"]:
    df[c] = pd.to_datetime(df[c], errors="coerce")
df["Survival_Months"] = (df["Death_Date"] - df["Diagnosis_Date"]).dt.days/30.44
df = df[df["Death_Date"].notna()] 


num = ["Age","Weight","Height"]
cat = ["Cancer_Stage","Cancer_Type","Treatment_Type","Gender","Smoking_Status","Comorbidities"]
use = num + cat + ["Survival_Months"]
df = df[use].dropna()

X, y = df[num + cat], df["Survival_Months"].astype(float)


prep = ColumnTransformer([("num", StandardScaler(), num),
                          ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat)])
pipe = Pipeline([("prep", prep), ("linreg", LinearRegression())])


Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=42)
cv = KFold(n_splits=5, shuffle=True, random_state=42)
cv_mae = -cross_val_score(pipe, Xtr, ytr, cv=cv, scoring="neg_mean_absolute_error")
cv_rmse = np.sqrt(-cross_val_score(pipe, Xtr, ytr, cv=cv, scoring="neg_mean_squared_error"))
cv_r2 = cross_val_score(pipe, Xtr, ytr, cv=cv, scoring="r2")
print(f"CV 5-fold → MAE {cv_mae.mean():.2f}±{cv_mae.std():.2f} | RMSE {cv_rmse.mean():.2f}±{cv_rmse.std():.2f} | R² {cv_r2.mean():.3f}±{cv_r2.std():.3f}")

pipe.fit(Xtr, ytr)
yp = pipe.predict(Xte)
mae = mean_absolute_error(yte, yp); rmse = np.sqrt(mean_squared_error(yte, yp)); r2 = r2_score(yte, yp)
ybase = np.repeat(ytr.mean(), len(yte))
mae_b = mean_absolute_error(yte, ybase); rmse_b = np.sqrt(mean_squared_error(yte, ybase))
print(f"Test → MAE {mae:.2f} | RMSE {rmse:.2f} | R² {r2:.3f}")
print(f"Baseline → MAE {mae_b:.2f} | RMSE {rmse_b:.2f}")


ohe = pipe.named_steps["prep"].named_transformers_["cat"]
feat_names = np.r_[num, ohe.get_feature_names_out(cat)]
coefs = pipe.named_steps["linreg"].coef_
top = pd.DataFrame({"feature": feat_names, "coef": coefs, "abs": np.abs(coefs)}).sort_values("abs", ascending=False).head(10)
print("\nTop 10 coefficients (abs):\n", top[["feature","coef"]].to_string(index=False))


CV 5-fold → MAE 7.89±0.33 | RMSE 9.33±0.35 | R² -0.074±0.064
Test → MAE 7.83 | RMSE 9.12 | R² 0.045
Baseline → MAE 8.12 | RMSE 9.36

Top 10 coefficients (abs):
                              feature      coef
                    Cancer_Type_Lung -2.972016
                   Cancer_Type_Liver -2.647433
                Cancer_Type_Leukemia  2.352115
                        Gender_Other -2.198580
          Comorbidities_Hypertension -1.543359
                Cancer_Type_Prostate  1.487297
                       Gender_Female  1.124511
                         Gender_Male  1.074069
Comorbidities_Cardiovascular Disease  1.033964
            Treatment_Type_Radiation  0.832115
